In [1]:
import openai
import json
import requests


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

def read_website_content(url):    
    """Reads the content on a given website.

    Args:
    url: The URL of the website to read.

    Returns:
    The content of the website.
    """

    response = requests.get(url)
    body = response.text
    if len(body)>30000:
        body = body[:30000]
    return body


def get_urls_inside_website(query: str,website_content: str):
    """Returns a list with all the relevant urls for a given query inside a given website."""
    
    prompt = f"Given this query: {query}, find all the relevant urls inside this website: {website_content}. The output should ONLY be a Python list with all the urls found."
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages = [{"role": "user", "content": prompt}]
    )
    response_message = response["choices"][0]["message"]
    
    return response_message
    

def run_conversation(query: str):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": query}]
    functions = [
        {
            "name": "read_website_content",
            "description": "Find the relevant information in a website",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The url of the website to read",
                    },
                },
                "required": ["url"],
            },
        },
        {
            "name": "get_urls_inside_website",
            "description": "Find the relevant urls in a website given a task",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The task description from user",
                    "website_content":{
                        "type": "string",
                        "description": "The content of the website to be searched"}
                    },
                },
                "required": ["query", "website_content"],
            },
        },
        
        
        
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "read_website_content": read_website_content,
            "get_urls_inside_website": get_urls_inside_website,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        if function_name not in available_functions.keys():
            raise Exception(f"Function {function_name} not found")
        
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        
        if function_name == "read_website_content":
            function_response = fuction_to_call(
                url=function_args.get("url"),
            )
        
        elif function_name == "get_urls_inside_website":
            function_response = fuction_to_call(
                query=function_args.get("query"),
                website_content=function_args.get("website_content")
            )
        
        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

print(run_conversation("Learn how to perform image classification with keras given this initial resource containing relevant urls: https://keras.io/examples/ "))

SSLError: HTTPSConnectionPool(host='keras.io', port=443): Max retries exceeded with url: /examples/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

## OpenAI Function Calling with Langchain

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
system_message = SystemMessage(content="You are very powerful programming and Machine Learning Assistant.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.run("")

# References
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb
- https://openai.com/blog/function-calling-and-other-api-updates
- https://platform.openai.com/docs/guides/gpt/function-calling